## Imports

In [1]:
import pandas as pd
import numpy as np

# CNN
import keras

# Graph
import seaborn as sns

# Model
from sklearn.model_selection import train_test_split

# AWS
import boto3

Using TensorFlow backend.


## Flow Data Into Keras From Directory

In [2]:
# Alters Imgaes
image_gen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0.0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None)

In [3]:
# Taking data and classes from the data directory
image_gen = image_gen.flow_from_directory(
    "Image-Data",  # Location of data/images
    target_size=(300, 300),  # Image pixels
    color_mode='rgb',  # Color scale
    classes=None,
    class_mode='categorical',
    batch_size=5_000,  # Amount of chosen data
    shuffle=True,
    seed=123,  # Keeps same results
    save_to_dir=None,
    save_prefix='',
    save_format='png',  # File type
    follow_links=False,
    interpolation='nearest')

Found 3913 images belonging to 2 classes.


In [4]:
# Number of classes and what classes there are
image_gen.class_indices

{'Normal': 0, 'Pneumonia': 1}

## Images and Labels

In [10]:
images, labels = next(image_gen)
images.shape, labels.shape

MemoryError: 

## Showing Images

In [ ]:
def show_image(n):
    """
    n represents the image number
    you want to see.
    images is a tensor (a bunch of array)
    and entering the number will show you
    which tensor/image you chose.
    The labels are connected to each image,
    choosing the same image number for labels
    will show you the label/class that goes
    with the image.
    """
    
    print(f"Classes:{image_gen.class_indices}")  # Prints the two classes we have along with their indices
    plt.imshow(images[n])  # Shows our chosen image
    print(labels[n])  # Shows which label/class the chosen image is
    
show_image(80)

## Test Split

In [ ]:
# Obtaining test data to use as unseen data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=24)

## Train and Validation Split

In [ ]:
# Obtaining training and validation sets to train the neural network
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=24)

## Convolutional Neural Network

In [ ]:
# Convenlutional Neural Network Model
cnn = keras.models.Sequential()

# 1st - convolutional layer
cnn.add(keras.layers.Conv2D(64, (3, 3),
                      activation='relu',
                      input_shape=(300, 300,  3),
                      padding='SAME'))
cnn.add(keras.layers.MaxPooling2D((2, 2)))

# 2nd - convolutional layer
cnn.add(keras.layers.Conv2D(32, (3, 3),
                            activation='relu', 
                            padding='SAME'))

cnn.add(keras.layers.Conv2D(32, (3, 3),
                            activation='relu', 
                            padding='SAME'))


# 3rd - convolutional layer
cnn.add(keras.layers.Conv2D(15, (3, 3), 
                            activation='relu', 
                            padding='SAME'))
cnn.add(keras.layers.MaxPooling2D((2, 2)))

# Flatten images
cnn.add(keras.layers.Flatten())

cnn.add(keras.layers.Dense(15, activation='relu'))  # 5th - dense layer
cnn.add(keras.layers.Dense(3, activation='softmax'))  # 6th - dense layer

# Producing Neural Network
cnn.compile(loss='binary_crossentropy',  # Loss function
            optimizer="sgd",  # Gradient Descent
            metrics=['acc'])  # Accuracy score

## Callbacks for CNN

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='loss', factor=0.1, patience=20,
    verbose=0, mode='auto', min_lr=0)

In [ ]:
# If there are problems with input data then it will stop it
nan_problem = keras.callbacks.TerminateOnNaN()

In [ ]:
# Stops Epochs when modeling stops to improve
early_stop = keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=20,
    verbose=0, mode='auto')

## Content Summary of CNN

In [ ]:
print(cnn.summary())

## Training and Testing The CNN

In [ ]:
# Fitting the model to the data to train/learn to predict.
cnn1 = cnn.fit(X_train,
               y_train,
               validation_data=(X_val, y_val),
               epochs=80,
               batch_size=10,
               callbacks=[early_stop, nan_problem, reduce_lr])

In [ ]:
# Getting the accuracy score using train and test
results_train = cnn.evaluate(X_train, y_train)
results_test = cnn.evaluate(X_test, y_test)
print(results_train, results_test)